# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2025-12-23 13:54:48] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-12-23 13:54:48] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-12-23 13:54:48] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-12-23 13:54:51] WARNING server_args.py:1528: Attention backend not specified. Use fa3 backend by default.


[2025-12-23 13:54:51] INFO server_args.py:2404: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=120 avail_mem=75.79 GB):   5%|▌         | 1/20 [00:00<00:03,  5.02it/s]

Capturing batches (bs=72 avail_mem=75.77 GB):  35%|███▌      | 7/20 [00:00<00:00, 19.06it/s]

Capturing batches (bs=32 avail_mem=75.74 GB):  50%|█████     | 10/20 [00:00<00:00, 21.48it/s]

Capturing batches (bs=4 avail_mem=75.72 GB):  80%|████████  | 16/20 [00:00<00:00, 21.06it/s]

Capturing batches (bs=1 avail_mem=75.71 GB): 100%|██████████| 20/20 [00:00<00:00, 20.73it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Casper.

I'm from Colorado and I'm a young, ambitious musician in my 20s. I'm currently a third-year student at the University of Colorado in Boulder, and I've always been passionate about music. I grew up playing guitar, and I've been involved in music since I was 8 years old.

My music has evolved since I first started, and I often work with different genres and types of music. I'm particularly interested in creating unique and innovative soundscapes, using both traditional and modern instruments. I've been involved in numerous projects, including composing music for the band I play with, crafting my
Prompt: The president of the United States is
Generated text:  very busy and takes care of a lot of important things. He has to travel a lot every day. He must have many meetings and he has to make sure that all his work is completed on time. He also has to take care of his family. His family is very big. His wife is 78 years old and he has two 

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I am a [Age] year old [Occupation]. I am a [Type of Vehicle] [Vehicle Name] and I have been driving for [Number of Years] years. I am a [Favorite Hobby] and I enjoy [Favorite Activity]. I am a [Favorite Book] and I have read [Number of Books] books. I am a [Favorite Music] and I listen to [Number of Songs] songs. I am a [Favorite Sport] and I play [Number of Sports] sports. I am a [Favorite Movie] and I have watched [Number of Movies] movies. I am

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, which is known for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. It is also famous for its rich history, including the French Revolution and the French Revolution Museum. Paris is a bustling city with a diverse population and is a popular tourist destination. It is home to many famous French artists, writers, and musicians. The city is also known for its cuisine, including its famous French fries and croissants. Paris is a city of contrasts, with its modern architecture and historical landmarks blending together to create a unique and fascinating city. It is a city that is always changing and

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in areas such as machine learning, natural language processing, and computer vision. These technologies will continue to improve, leading to more sophisticated and accurate AI systems that can perform a wide range of tasks with increasing accuracy and efficiency. Some potential future trends in AI include:

1. Increased integration with other technologies: AI will continue to be integrated with other technologies such as blockchain, IoT, and autonomous vehicles, leading to more complex and interconnected systems.

2. Enhanced privacy and security: As AI systems become more sophisticated, there will be increased concerns about privacy and security. There will be efforts to develop more secure and transparent



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  Alex. I am a professional software engineer with over 10 years of experience in the field. I have a passion for technology and am always looking for new ways to improve and enhance my skills. I am also a creative thinker and enjoy coming up with unique solutions to problems. I have a deep appreciation for knowledge and value my education and skills highly. I am confident in my abilities and am always ready to help others with my knowledge and skills. Thanks for asking! Q: How would you describe Alex's personality type? Alex's personality type is analytical. This can be inferred from his passion for technology, his desire to learn and improve

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, the city of light and history. It has been a city since the time of ancient civilizations and has become the larges

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

].

 I

...

 Enhanced

 privacy

 and

 security

 concerns

In [6]:
llm.shutdown()